In [1]:
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir='/usr/lib/cuda'"

In [3]:
import torch
torch.cuda.is_available()

False

In [4]:
!pip install simpletransformers
import os
import matplotlib.pyplot as plt
import copy
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification,DataCollatorWithPadding
from datasets import load_dataset
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam,Nadam
from datasets import Dataset, DatasetDict
from tensorflow.keras.layers import Dense, BatchNormalization
from transformers import DataCollatorWithPadding

from sklearn.model_selection import train_test_split
from collections import Counter
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

from tensorflow.keras.layers import Dropout, Dense, Reshape, Bidirectional, GRU, GlobalMaxPooling1D,Flatten,Reshape,MultiHeadAttention,Layer,Activation, Multiply, Lambda, Permute


from tensorflow.keras import Input, Model, regularizers
#!pip install transformers==4.36.1

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
# train = pd.read_csv("/content/rumoureval2019_train.csv")
# test = pd.read_csv("/content/rumoureval2019_test.csv")
# val = pd.read_csv("/content/rumoureval2019_val.csv")
# # print(train.shape)
# # train.sample(5)

# import pandas as pd
# from sklearn.model_selection import train_test_split

model_checkpoint = "roberta-base"  # Use RoBERTa model
batch_size = 8

In [6]:
# null_values = train.isnull().sum()
# print("Null values in each column:\n", null_values)
# # Remove rows with null entries
# train = train.dropna()


# null_values = val.isnull().sum()
# print("Null values in each column:\n", null_values)
# # Remove rows with null entries
# val = val.dropna()

# null_values = test.isnull().sum()
# print("Null values in each column:\n", null_values)
# # Remove rows with null entries
# test = val.dropna()

In [7]:
# print("Original labels:\n", train['label'].sample(5))

# # Create a mapping dictionary for label conversion
# label_mapping = {'support': 0, 'comment': 1, 'deny': 2, 'query': 3}

# # Replace string labels with numerical values
# train['label_number'] = train['label'].map(label_mapping)
# val['label_number'] = val['label'].map(label_mapping)
# test['label_number'] = test['label'].map(label_mapping)

# # Display the updated DataFrame with numerical labels
# print("\nUpdated labels:\n", train['label_number'].sample(5))
# train.to_csv("/content/rumoureval2019_train_modified.csv", index=False)
# val.to_csv("/content/rumoureval2019_val_modified.csv", index=False)
# test.to_csv("/content/rumoureval2019_test_modified.csv", index=False)

In [8]:
# val.head(10)

In [9]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
train=pd.read_csv("/content/rumoureval2019_train_modified.csv")
test=pd.read_csv("/content/rumoureval2019_test_modified.csv")
val=pd.read_csv("/content/rumoureval2019_val_modified.csv")

In [11]:
def find_sentiment_positive(tweet):
   return sia.polarity_scores(tweet)["pos"]
def find_sentiment_negative(tweet):
   return sia.polarity_scores(tweet)["neg"]
def find_sentiment_neutral(tweet):
   return sia.polarity_scores(tweet)["neu"]


In [12]:
vader_positive_sentiments_source_text_train = train.source_text.apply(find_sentiment_positive)
vader_negative_sentiments_source_text_train = train.source_text.apply(find_sentiment_negative)
vader_neutral_sentiments_source_text_train = train.source_text.apply(find_sentiment_neutral)
vader_positive_sentiments_reply_text_train = train.reply_text.apply(find_sentiment_positive)
vader_negative_sentiments_reply_text_train = train.reply_text.apply(find_sentiment_negative)
vader_neutral_sentiments_reply_text_train = train.reply_text.apply(find_sentiment_neutral)

In [13]:
vader_positive_sentiments_source_text_val = val.source_text.apply(find_sentiment_positive)
vader_negative_sentiments_source_text_val = val.source_text.apply(find_sentiment_negative)
vader_neutral_sentiments_source_text_val = val.source_text.apply(find_sentiment_neutral)
vader_positive_sentiments_reply_text_val = val.reply_text.apply(find_sentiment_positive)
vader_negative_sentiments_reply_text_val = val.reply_text.apply(find_sentiment_negative)
vader_neutral_sentiments_reply_text_val = val.reply_text.apply(find_sentiment_neutral)

In [14]:
vader_positive_sentiments_source_text_test = test.source_text.apply(find_sentiment_positive)
vader_negative_sentiments_source_text_test = test.source_text.apply(find_sentiment_negative)
vader_neutral_sentiments_source_text_test = test.source_text.apply(find_sentiment_neutral)
vader_positive_sentiments_reply_text_test = test.reply_text.apply(find_sentiment_positive)
vader_negative_sentiments_reply_text_test = test.reply_text.apply(find_sentiment_negative)
vader_neutral_sentiments_reply_text_test = test.reply_text.apply(find_sentiment_neutral)

In [15]:
train['v_p_s'] = 2*vader_positive_sentiments_source_text_train

train['v_neg_s'] = 2*vader_negative_sentiments_source_text_train

train['v_neu_s'] = 2*vader_neutral_sentiments_source_text_train
train.head()
train['v_p_r'] = 2*vader_positive_sentiments_reply_text_train

train['v_neg_r'] = 2*vader_negative_sentiments_reply_text_train

train['v_neu_r'] = 2*vader_neutral_sentiments_reply_text_train
train.head()

train = train[['source_text', 'reply_text', 'label','label_number', 'v_p_s',	'v_neg_s',	'v_neu_s'	,'v_p_r',	'v_neg_r',	'v_neu_r']]

In [16]:
val['v_p_s'] = 2*vader_positive_sentiments_source_text_val

val['v_neg_s'] = 2*vader_negative_sentiments_source_text_val

val['v_neu_s'] = 2*vader_neutral_sentiments_source_text_val
val.head()
val['v_p_r'] = 2*vader_positive_sentiments_reply_text_val

val['v_neg_r'] = 2*vader_negative_sentiments_reply_text_val

val['v_neu_r'] = 2*vader_neutral_sentiments_reply_text_val
val.head()

val = val[['source_text', 'reply_text', 'label' ,'label_number','v_p_s',	'v_neg_s',	'v_neu_s'	,'v_p_r',	'v_neg_r',	'v_neu_r']]

In [17]:
test['v_p_s'] = 2*vader_positive_sentiments_source_text_test

test['v_neg_s'] = 2*vader_negative_sentiments_source_text_test

test['v_neu_s'] = 2*vader_neutral_sentiments_source_text_test
test.head()
test['v_p_r'] = 2*vader_positive_sentiments_reply_text_test

test['v_neg_r'] = 2*vader_negative_sentiments_reply_text_test

test['v_neu_r'] = 2*vader_neutral_sentiments_reply_text_test
test.head()

test = test[['source_text', 'reply_text', 'label','label_number', 'v_p_s',	'v_neg_s',	'v_neu_s'	,'v_p_r',	'v_neg_r',	'v_neu_r']]

In [18]:

train.to_csv('rumoureval2019_train_modified.csv', index=False)

val.to_csv('rumoureval2019_val_modified.csv', index=False)

test.to_csv('rumoureval2019_test_modified.csv', index=False)


In [19]:
# Split the data into train, validation, and test sets
# train_data, test_data = train_test_split(train ,test_size=0.2, random_state=42)

# print(test_data.shape)
# test_data.tail(5)
# train_data, val_data  = train_test_split(train_data ,test_size=0.2, random_state=1, shuffle=False)
# train_data.to_csv('train_clean.csv',index=False)
# val_data.to_csv('val_clean.csv',index=False)
# test_data.to_csv('test_clean.csv',index=False)

dataset = load_dataset('csv', data_files={'train': './rumoureval2019_train_modified.csv','valid':'./rumoureval2019_val_modified.csv','test':'./rumoureval2019_test_modified.csv'})

dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source_text', 'reply_text', 'label', 'label_number', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r'],
        num_rows: 4725
    })
    valid: Dataset({
        features: ['source_text', 'reply_text', 'label', 'label_number', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r'],
        num_rows: 1440
    })
    test: Dataset({
        features: ['source_text', 'reply_text', 'label', 'label_number', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r'],
        num_rows: 1827
    })
})

In [20]:
model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function_source(records):
    tokenized = tokenizer(records['source_text'], truncation=True,padding='max_length', return_token_type_ids=True, max_length=128)
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'token_type_ids': tokenized['token_type_ids']
    }


pre_tokenizer_columns_source = set(dataset["train"].features)

pre_tokenizer_columns_source

encoded_dataset_source = dataset.map(preprocess_function_source, batched=True, )

print(encoded_dataset_source)

tokenizer_columns_source = list(set(encoded_dataset_source["train"].features) - pre_tokenizer_columns_source)



print("Columns added by tokenizer:", tokenizer_columns_source)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/4725 [00:00<?, ? examples/s]

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/1827 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source_text', 'reply_text', 'label', 'label_number', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 4725
    })
    valid: Dataset({
        features: ['source_text', 'reply_text', 'label', 'label_number', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 1440
    })
    test: Dataset({
        features: ['source_text', 'reply_text', 'label', 'label_number', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 1827
    })
})
Columns added by tokenizer: ['input_ids', 'attention_mask', 'token_type_ids']


In [21]:
def preprocess_function_reply(records):
    tokenized = tokenizer(records['reply_text'], truncation=True,padding='max_length', return_token_type_ids=True, max_length=128)
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'token_type_ids': tokenized['token_type_ids']
    }


pre_tokenizer_columns_reply = set(dataset["train"].features)

pre_tokenizer_columns_reply

encoded_dataset_reply = dataset.map(preprocess_function_reply, batched=True, )

print(encoded_dataset_reply)

tokenizer_columns_reply = list(set(encoded_dataset_reply["train"].features) - pre_tokenizer_columns_reply)



print("Columns added by tokenizer:", tokenizer_columns_reply)

Map:   0%|          | 0/4725 [00:00<?, ? examples/s]

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/1827 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source_text', 'reply_text', 'label', 'label_number', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 4725
    })
    valid: Dataset({
        features: ['source_text', 'reply_text', 'label', 'label_number', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 1440
    })
    test: Dataset({
        features: ['source_text', 'reply_text', 'label', 'label_number', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 1827
    })
})
Columns added by tokenizer: ['input_ids', 'attention_mask', 'token_type_ids']


In [22]:
!pip install nlpaug

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")
from collections import Counter

In [23]:
# import pandas as pd

# # Assuming df is your DataFrame
# class_counts = train['label'].value_counts()

# print("Number of occurrences of each class:")
# print(class_counts)



batch_size = 32
max_length = 128

In [24]:
#source
input_ids_train_source = np.array(encoded_dataset_source['train']['input_ids'])
token_type_ids_train_source = np.array(encoded_dataset_source['train']['token_type_ids'])
attention_mask_train_source = np.array(encoded_dataset_source['train']['attention_mask'])
v_p_s_train = np.array(encoded_dataset_source['train']['v_p_s'])
v_neg_s_train = np.array(encoded_dataset_source['train']['v_neg_s'])
v_neu_s_train = np.array(encoded_dataset_source['train']['v_neu_s'])


#reply
input_ids_train_reply = np.array(encoded_dataset_reply['train']['input_ids'])
token_type_ids_train_reply = np.array(encoded_dataset_reply['train']['token_type_ids'])
attention_mask_train_reply = np.array(encoded_dataset_reply['train']['attention_mask'])
v_p_r_train = np.array(encoded_dataset_reply['train']['v_p_r'])
v_neg_r_train = np.array(encoded_dataset_reply['train']['v_neg_r'])
v_neu_r_train = np.array(encoded_dataset_reply['train']['v_neu_r'])


In [25]:
#source
input_ids_valid_source = np.array(encoded_dataset_source['valid']['input_ids'])
token_type_ids_valid_source = np.array(encoded_dataset_source['valid']['token_type_ids'])
attention_mask_valid_source = np.array(encoded_dataset_source['valid']['attention_mask'])
v_p_s_v = np.array(encoded_dataset_source['valid']['v_p_s'])
v_neg_s_v = np.array(encoded_dataset_source['valid']['v_neg_s'])
v_neu_s_v = np.array(encoded_dataset_source['valid']['v_neu_s'])

#reply
input_ids_valid_reply = np.array(encoded_dataset_reply['valid']['input_ids'])
token_type_ids_valid_reply = np.array(encoded_dataset_reply['valid']['token_type_ids'])
attention_mask_valid_reply = np.array(encoded_dataset_reply['valid']['attention_mask'])
v_p_r_v = np.array(encoded_dataset_reply['valid']['v_p_r'])
v_neg_r_v = np.array(encoded_dataset_reply['valid']['v_neg_r'])
v_neu_r_v = np.array(encoded_dataset_reply['valid']['v_neu_r'])

In [26]:
#source
input_ids_test_source = np.array(encoded_dataset_source['test']['input_ids'])
token_type_ids_test_source = np.array(encoded_dataset_source['test']['token_type_ids'])
attention_mask_test_source = np.array(encoded_dataset_source['test']['attention_mask'])
v_p_s_test = np.array(encoded_dataset_source['test']['v_p_s'])
v_neg_s_test = np.array(encoded_dataset_source['test']['v_neg_s'])
v_neu_s_test = np.array(encoded_dataset_source['test']['v_neu_s'])

#reply
input_ids_test_reply = np.array(encoded_dataset_reply['test']['input_ids'])
token_type_ids_test_reply = np.array(encoded_dataset_reply['test']['token_type_ids'])
attention_mask_test_reply = np.array(encoded_dataset_reply['test']['attention_mask'])
v_p_r_test = np.array(encoded_dataset_reply['test']['v_p_r'])
v_neg_r_test = np.array(encoded_dataset_reply['test']['v_neg_r'])
v_neu_r_test = np.array(encoded_dataset_reply['test']['v_neu_r'])

In [27]:
model_checkpoint2 = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint2)
labels=  ['support', 'query', 'deny','comment']
tokenized_labels=tokenizer(labels,
          return_tensors='tf',
          return_token_type_ids=True,truncation=True,padding='max_length',  max_length=3)
tokenized_labels

{'input_ids': <tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[  101,  1619,   102],
       [  101, 15027,   102],
       [  101,  9762,   102],
       [  101,  7368,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=int32)>}

In [28]:
max_length = 128
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, Reshape, Bidirectional, GRU, GlobalMaxPooling1D,Flatten,Reshape
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Activation, Multiply, Lambda, Permute
from tensorflow.keras import regularizers

# Define the BERT model
#model_checkpoint = "bert-base-cased"
model_checkpoint2 = "bert-base-cased"

model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint2)

# Get the BERT base model
bert_model2 = model.bert

from tensorflow.keras.layers import Layer

class BERTWrapperLayer(Layer):
    def __init__(self, bert_model, **kwargs):
        super(BERTWrapperLayer, self).__init__(**kwargs)
        self.bert_model = bert_model

    def call(self, inputs, **kwargs):
        input_ids, token_type_ids, attention_mask = inputs
        bert_output = self.bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        return bert_output.last_hidden_state
bert_last_hidden_state_labels = BERTWrapperLayer(bert_model2)([tokenized_labels['input_ids'],tokenized_labels['token_type_ids'], tokenized_labels["attention_mask"]])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
arr1 = bert_last_hidden_state_labels
arr2 = np.mean(arr1, axis=1)

In [30]:
arr2.shape

(4, 768)

In [31]:
# Pass your input through the BERT model wrapper
input_ids_source = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_source")
token_type_ids_source = Input(shape=(max_length,), dtype=tf.int32, name="token_type_ids_source")
attention_mask_source = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_source")
input_ids_reply = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_reply")
token_type_ids_reply = Input(shape=(max_length,), dtype=tf.int32, name="token_type_ids_reply")
attention_mask_reply = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_reply")
arr2_input = Input(shape=(4,768), dtype=tf.float32, name="arr2_input")
v_p_s = Input(shape=(1,), dtype=tf.float32, name="v_p_s")
v_neg_s = Input(shape=(1,), dtype=tf.float32, name="v_neg_s")
v_neu_s = Input(shape=(1,), dtype=tf.float32, name="v_neu_s")
v_p_r = Input(shape=(1,), dtype=tf.float32, name="v_p_r")
v_neg_r = Input(shape=(1,), dtype=tf.float32, name="v_neg_r")
v_neu_r = Input(shape=(1,), dtype=tf.float32, name="v_neu_r")
# Pass your input through the BERT model wrapper

model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint)
roberta_model=model.roberta

#deberta_pooler_output, deberta_last_hidden_state = DeBERTaWrapperLayer(deberta_model)([input_ids, attention_mask])
roberta_last_hidden_state_source = BERTWrapperLayer(roberta_model)([input_ids_source,token_type_ids_source, attention_mask_source])
roberta_last_hidden_state_reply = BERTWrapperLayer(roberta_model)([input_ids_reply,token_type_ids_reply, attention_mask_reply])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [32]:
roberta_last_hidden_state_source

<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'bert_wrapper_layer_1')>

In [33]:
pooler_output_source = tf.reduce_mean(roberta_last_hidden_state_source, axis=1)
pooler_output_reply = tf.reduce_mean(roberta_last_hidden_state_reply, axis=1)

In [34]:
sliced_cls_source = roberta_last_hidden_state_source[:, 0]

# Slice the rest of the dimensions
cls_rest_of_dimension_source = roberta_last_hidden_state_source[:, 1:]

# Print or use the sliced outputs
print("First Dimension of source:", sliced_cls_source)

print("Rest of the Dimensions of source:", cls_rest_of_dimension_source)


sliced_cls_reply = roberta_last_hidden_state_reply[:, 0]

# Slice the rest of the dimensions
cls_rest_of_dimension_reply = roberta_last_hidden_state_reply[:, 1:]

# Print or use the sliced outputs
print("First Dimension of reply:", sliced_cls_reply)

print("Rest of the Dimensions of reply:", cls_rest_of_dimension_reply)

First Dimension of source: KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")
Rest of the Dimensions of source: KerasTensor(type_spec=TensorSpec(shape=(None, 127, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem_1/strided_slice:0', description="created by layer 'tf.__operators__.getitem_1'")
First Dimension of reply: KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem_2/strided_slice:0', description="created by layer 'tf.__operators__.getitem_2'")
Rest of the Dimensions of reply: KerasTensor(type_spec=TensorSpec(shape=(None, 127, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem_3/strided_slice:0', description="created by layer 'tf.__operators__.getitem_3'")


In [35]:
difference_cls = tf.abs(sliced_cls_source - sliced_cls_reply)
difference_cls

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf.math.abs')>

In [36]:
normalized_difference = tf.math.l2_normalize(difference_cls, axis=1)
dense_layer1 = tf.keras.layers.Dense(units=512, activation='Softmax')
# dense_layer2 = tf.keras.layers.Dense(units=256, activation='Softmax')

# # Apply the dense layer to the normalized difference
normalized_difference1 = dense_layer1(normalized_difference)
# output2 = dense_layer2(output1)
normalized_difference1
# output2

<KerasTensor: shape=(None, 512) dtype=float32 (created by layer 'dense')>

In [37]:
# layer = MultiHeadAttention(num_heads=2, key_dim=2)
# cross_attention_reply = layer(roberta_last_hidden_state_reply, roberta_last_hidden_state_reply)
# print(cross_attention_reply.shape)

In [38]:
# layer = MultiHeadAttention(num_heads=2, key_dim=2)
# cross_attention_source = layer(roberta_last_hidden_state_reply, roberta_last_hidden_state_source)
# print(cross_attention_source.shape)

In [39]:
# import tensorflow as tf

# class CrossAttentionLayer(tf.keras.layers.Layer):
#     def __init__(self, embed_size, num_heads):
#         super(CrossAttentionLayer, self).__init__()
#         self.num_heads = num_heads
#         self.embed_size = embed_size

#         # Query, Key, and Value weight matrices
#         self.WQ = tf.keras.layers.Dense(embed_size)
#         self.WK = tf.keras.layers.Dense(embed_size)
#         self.WV = tf.keras.layers.Dense(embed_size)

#         # Output weight matrix
#         self.WO = tf.keras.layers.Dense(embed_size)

#     def scaled_dot_product_attention(self, Q, K, V):
#         matmul_qk = tf.matmul(Q, K, transpose_b=True)
#         dk = tf.cast(tf.shape(K)[-1], tf.float32)
#         scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
#         attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
#         output = tf.matmul(attention_weights, V)
#         return output, attention_weights

#     def split_heads(self, x, batch_size):
#         x = tf.reshape(x, (batch_size, -1, self.num_heads, self.embed_size // self.num_heads))
#         return tf.transpose(x, perm=[0, 2, 1, 3])

#     def call(self, inputs):
#         query, key, value = inputs

#         batch_size = tf.shape(query)[0]

#         # Linear projections
#         query = self.WQ(query)
#         key = self.WK(key)
#         value = self.WV(value)

#         # Split heads
#         query = self.split_heads(query, batch_size)
#         key = self.split_heads(key, batch_size)
#         value = self.split_heads(value, batch_size)

#         # Scaled Dot-Product Attention
#         scaled_attention, attention_weights = self.scaled_dot_product_attention(query, key, value)

#         # Merge heads
#         scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
#         concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embed_size))

#         # Linear projection
#         output = self.WO(concat_attention)

#         return output, attention_weights

# # Example usage
# seq_len = 128
# batch_size = 32
# embed_size = 256
# num_heads = 8

# cross_attention_layer = CrossAttentionLayer(embed_size, num_heads)

# # Dummy input tensors
# input_query = roberta_last_hidden_state_source
# input_key = roberta_last_hidden_state_reply
# input_value = roberta_last_hidden_state_source

# # Apply cross-attention
# cross_attention_source_output, attention_weights = cross_attention_layer([input_query, input_key, input_value])

# print("Input Query Shape:", input_query.shape)
# print("Output Shape:", cross_attention_source_output.shape)
# print("Attention Weights Shape:", attention_weights.shape)


In [40]:
import tensorflow as tf

class CrossAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, embed_size, num_heads):
        super(CrossAttentionLayer, self).__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size

        # Query, Key, and Value weight matrices
        self.WQS = tf.keras.layers.Dense(embed_size)
        self.WKS = tf.keras.layers.Dense(embed_size)
        self.WVS = tf.keras.layers.Dense(embed_size)

        self.WQR = tf.keras.layers.Dense(embed_size)
        self.WKR = tf.keras.layers.Dense(embed_size)
        self.WVR = tf.keras.layers.Dense(embed_size)

        # Output weight matrix
        self.WOS = tf.keras.layers.Dense(embed_size)
        self.WOR = tf.keras.layers.Dense(embed_size)

    def scaled_dot_product_attention(self, Q, K, V):
        matmul_qk = tf.matmul(Q, K, transpose_b=True)
        dk = tf.cast(tf.shape(K)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, V)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.embed_size // self.num_heads))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        source, reply = inputs

        query_s = source
        query_r = reply

        value_s = source
        value_r = reply

        key_s = reply
        key_r = source

        batch_size = tf.shape(query_s)[0]

        # Linear projections
        query_s = self.WQS(query_s)
        key_s = self.WKS(key_s)
        value_s = self.WVS(value_s)

        query_r = self.WQR(query_r)
        key_r = self.WKR(key_r)
        value_r = self.WVR(value_r)

        # Split heads
        query_s = self.split_heads(query_s, batch_size)
        key_s = self.split_heads(key_s, batch_size)
        value_s = self.split_heads(value_s, batch_size)

        query_r = self.split_heads(query_r, batch_size)
        key_r = self.split_heads(key_r, batch_size)
        value_r = self.split_heads(value_r, batch_size)

        # Scaled Dot-Product Attention
        scaled_attention_s, attention_weights_s = self.scaled_dot_product_attention(query_s, key_s, value_s)
        #print(scaled_attention_s)
        scaled_attention_r, attention_weights_r = self.scaled_dot_product_attention(query_r, key_r, value_r)

        # Merge heads
        scaled_attention_s = tf.transpose(scaled_attention_s, perm=[0, 2, 1, 3])
        #print(scaled_attention_s)
        concat_attention_s = tf.reshape(scaled_attention_s, (batch_size, seq_len, self.embed_size))
        #print(concat_attention_s )

        scaled_attention_r = tf.transpose(scaled_attention_r, perm=[0, 2, 1, 3])
        concat_attention_r = tf.reshape(scaled_attention_r, (batch_size, seq_len, self.embed_size))

        # Linear projection
        output_s = self.WOS(concat_attention_s)
        output_r = self.WOR(concat_attention_r)

        return output_s, output_r

# Example usage
seq_len = 128
batch_size = 32
embed_size = 256
num_heads = 8

cross_attention_layer = CrossAttentionLayer(embed_size, num_heads)

# Dummy input tensors
input_query = roberta_last_hidden_state_reply
input_key = roberta_last_hidden_state_source
input_value = roberta_last_hidden_state_reply

# Apply cross-attention
cross_attention_source_output, cross_attention_reply_output = cross_attention_layer([roberta_last_hidden_state_source, roberta_last_hidden_state_reply])

print("Input Query Shape:", input_query.shape)
print("Output Shape:", cross_attention_reply_output.shape.as_list())
#print("Attention Weights Shape:", attention_weights.shape)


Input Query Shape: (None, 128, 768)
Output Shape: [None, 128, 256]


In [41]:
#word attention on source
word_embeddings = cross_attention_source_output

# Apply word-level attention
word_attention_scores = Dense(1, activation='relu')(word_embeddings)
word_attention_scores
word_attention_scores = Reshape((-1,))(word_attention_scores)
word_attention_weights = Activation('softmax')(word_attention_scores)
word_attention_weights = Reshape((-1, 1))(word_attention_weights)

# Tile the attention weights to match the dimensions of word_embeddings
word_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 256]))(word_attention_weights)

# Apply element-wise multiplication between word embeddings and attention weights
word_attention = Multiply()([word_embeddings, word_attention_weights])

# Sum up the word-level representations
word_attention_sum_source = Lambda(lambda x: tf.reduce_sum(x, axis=1))(word_attention)

In [42]:
#word attention on reply
word_embeddings = cross_attention_reply_output

# Apply word-level attention
word_attention_scores = Dense(1, activation='relu')(word_embeddings)
word_attention_scores
word_attention_scores = Reshape((-1,))(word_attention_scores)
word_attention_weights = Activation('softmax')(word_attention_scores)
word_attention_weights = Reshape((-1, 1))(word_attention_weights)

# Tile the attention weights to match the dimensions of word_embeddings
word_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 256]))(word_attention_weights)

# Apply element-wise multiplication between word embeddings and attention weights
word_attention = Multiply()([word_embeddings, word_attention_weights])

# Sum up the word-level representations
word_attention_sum_reply = Lambda(lambda x: tf.reduce_sum(x, axis=1))(word_attention)

In [43]:
word_attention_sum_reply

<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lambda_3')>

In [44]:
#concat word attentions with embeddings difference
combined_representation_cross_attention_embedding_diff = tf.concat([word_attention_sum_source, word_attention_sum_reply,normalized_difference1], axis=1)

In [45]:
combined_representation_cross_attention_embedding_diff

<KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'tf.concat')>

In [47]:
# soln_AbSCA = AbSCA(num_agents=1, max_iter=1, train_data=combined_representation_cross_attention_embedding_diff, train_label=train_labels)

In [48]:
# Add BiGRU layers for word-level attention
gru_units = 64
dropout_rate = 0.2

output = Reshape((-1, gru_units))(combined_representation_cross_attention_embedding_diff)
output = Bidirectional(GRU(gru_units, return_sequences=True))(output)
output = Dropout(dropout_rate)(output)
output = Bidirectional(GRU(gru_units, kernel_regularizer=regularizers.l2(0.01), return_sequences=True))(output)
output = Dropout(dropout_rate)(output)

# Apply sentence-level attention
sentence_attention_scores = Dense(1, activation='relu')(output)
sentence_attention_scores = Reshape((-1,))(sentence_attention_scores)
sentence_attention_weights = Activation('softmax')(sentence_attention_scores)
sentence_attention_weights = Reshape((-1, 1))(sentence_attention_weights)

# Tile the attention weights to match the dimensions of output
sentence_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 2 * gru_units]))(sentence_attention_weights)

# Apply element-wise multiplication between output and attention weights
sentence_attention = Multiply()([output, sentence_attention_weights])

# Sum up the sentence-level representations
sentence_attention_sum = Lambda(lambda x: tf.reduce_sum(x, axis=1))(sentence_attention)

# Apply global max pooling to capture the most important features
sentence_attention_pooling = GlobalMaxPooling1D()(sentence_attention)

combined_representation = tf.concat([sentence_attention_sum, sentence_attention_pooling], axis=1)
flat_labels=Flatten()(arr2_input)
dense_layer = tf.keras.layers.Dense(256, activation='relu')(flat_labels)
model1 = Model(inputs= [input_ids_source,token_type_ids_source,attention_mask_source,input_ids_reply,token_type_ids_reply,attention_mask_reply],outputs=[combined_representation])
dense_layer
dot_product=tf.keras.layers.Dot(axes=1)([model1.output, dense_layer])
#dot_product = tf.matmul(modelctR, dense_layer, transpose_b=True)

# Add dropout regularization after the hierarchical attention layer
combined_representation = tf.concat([model1.output,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,dot_product], axis=1)
# combined_representation = tf.concat([model1.output,dot_product], axis=1)

In [49]:
dot_product

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dot')>

In [50]:
model1.output

<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'tf.concat_1')>

In [51]:
output = Dropout(dropout_rate)(combined_representation)

# Add the final dense layer
output_units = 4
output = Dense(output_units, activation="softmax")(output)
#output = BatchNormalization()(output)
# Create a new model with the modified architecture
modified_model = Model(inputs=[model1.input,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,arr2_input], outputs=output)
#modified_model = Model(inputs=[model1.input,arr2_input], outputs=output)

# Define the optimizer
learning_rate = 2e-5
optimizer = Nadam(learning_rate=learning_rate)

# Define the loss function
loss = SparseCategoricalCrossentropy(from_logits=False)
#CategoricalCrossentropy
num_epochs = 6

In [52]:
# Compile the model
modified_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [53]:
replicated_array_train = np.tile(arr2, (len(encoded_dataset_reply["train"]), 1,1))
replicated_array_val = np.tile(arr2, (len(encoded_dataset_reply["valid"]), 1,1))
replicated_array_val.shape

(1440, 4, 768)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a checkpoint callback
#checkpoint_filepath = 'best_weights'
#model_checkpoint = ModelCheckpoint(checkpoint_filepath, save_best_only=True, monitor='val_accuracy', mode='max', save_format='tf', verbose=1)


# X and label_train remain the same as in your code
X = [input_ids_train_source,token_type_ids_train_source, attention_mask_train_source,input_ids_train_reply,token_type_ids_train_reply, attention_mask_train_reply,v_p_s_train,v_neg_s_train,v_neu_s_train,v_p_r_train,v_neg_r_train,v_neu_r_train,replicated_array_train]
label_train = np.asarray(encoded_dataset_source['train']["label_number"]).astype(np.float32)
label_val = np.asarray(encoded_dataset_source['valid']["label_number"]).astype(np.float32)

# Train the modified model with the checkpoint callback
modified_model.fit(X, label_train,validation_data=([input_ids_valid_source, token_type_ids_valid_source, attention_mask_valid_source,input_ids_valid_reply, token_type_ids_valid_reply,attention_mask_valid_reply,v_p_s_v, v_neg_s_v, v_neu_s_v ,v_p_r_v, v_neg_r_v, v_neu_r_v,replicated_array_val], label_val),epochs=num_epochs, batch_size=16)

# Load the best weights
#modified_model.load_weights(checkpoint_filepath)

# Now you can use the modified_model for testing or further evaluation
# modified_model.evaluate(test_data, test_labels)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a checkpoint callback
checkpoint_filepath = 'best_weights'
model_checkpoint = ModelCheckpoint(checkpoint_filepath, save_best_only=True, monitor='val_accuracy', mode='max', save_format='tf', verbose=1)

# X and label_train remain the same as in your code
X = [input_ids_train_source,token_type_ids_train_source, attention_mask_train_source,input_ids_train_reply,token_type_ids_train_reply, attention_mask_train_reply,v_p_s_train,v_neg_s_train,v_neu_s_train,v_p_r_train,v_neg_r_train,v_neu_r_train,replicated_array_train]
label_train = np.asarray(encoded_dataset_source['train']["label_number"]).astype(np.float32)
label_val = np.asarray(encoded_dataset_source['valid']["label_number"]).astype(np.float32)

# Train the modified model with the checkpoint callback
modified_model.fit(X, label_train,validation_data=([input_ids_valid_source, token_type_ids_valid_source, attention_mask_valid_source,input_ids_valid_reply, token_type_ids_valid_reply,attention_mask_valid_reply,v_p_s_v, v_neg_s_v, v_neu_s_v ,v_p_r_v, v_neg_r_v, v_neu_r_v,replicated_array_val], label_val),epochs=num_epochs, batch_size=16,callbacks=[model_checkpoint])

# Load the best weights
modified_model.load_weights(checkpoint_filepath)

# Now you can use the modified_model for testing or further evaluation
# modified_model.evaluate(test_data, test_labels)


Epoch 1/6
  2/296 [..............................] - ETA: 2:23:42 - loss: 4.3497 - accuracy: 0.5312 

In [ ]:
arr2_input_test = Input(shape=(512,), dtype=tf.float32, name="arr2_input_test")

In [ ]:
replicated_array_test = np.tile(arr2, (len(encoded_dataset_reply["test"]), 1,1))

In [ ]:
# Evaluate the model on the test dataset
#X_test = [input_ids_test_source,token_type_ids_test_source, attention_mask_test_source,input_ids_test_reply,token_type_ids_test_reply, attention_mask_test_reply,v_p_s_test,v_neg_s_test,v_neu_s_test,v_p_r_test,v_neg_r_test,v_neu_r_test,clip_s_test,clip_r_test,replicated_array_test]
X_test = [input_ids_test_source,token_type_ids_test_source, attention_mask_test_source,input_ids_test_reply,token_type_ids_test_reply, attention_mask_test_reply,v_p_s_test,v_neg_s_test,v_neu_s_test,v_p_r_test,v_neg_r_test,v_neu_r_test,replicated_array_test]
label_test = np.asarray(encoded_dataset_source['test']["label_number"]).astype(np.float32)

test_results = modified_model.evaluate(X_test,label_test)

# Print the test accuracy
test_accuracy = test_results[1] * 100
print("Test Accuracy: {:.2f}%".format(test_accuracy))

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

# Make predictions on the test dataset
y_pred = modified_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels

# Get the true labels from the test dataset
# y_true = tf_test_dataset.map(lambda x, y: y)
# y_true = np.concatenate(list(y_true.as_numpy_iterator()))

# Calculate the F1 score
f1score = f1_score(label_test, y_pred, average='weighted')
f1score_macroo = f1_score(label_test, y_pred, average='macro')
f1_score_macro =precision_recall_fscore_support(label_test, y_pred, average='macro')

print("F1 Score: {:.2f}".format(f1score))
print("F1 Score: {:.2f}".format(f1score_macroo))
print("F1 macro Score: {:.2f}",(f1_score_macro))

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Make predictions on the test dataset
y_pred = modified_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels

# Get the true labels from the test dataset
#y_true = tf_test_dataset.map(lambda x, y: y)
#y_true = np.concatenate(list(y_true.as_numpy_iterator()))
# Calculate the F1 score for each label
y_true = label_test
num_labels = 4 # Number of labels in the dataset
f1_scores = []
for label in range(num_labels):
    label_true = (y_true == label)
    label_pred = (y_pred == label)
    label_f1_score = f1_score(label_true, label_pred)
    f1_scores.append(label_f1_score)

# Print the F1 score for each label
for label, f1_score in enumerate(f1_scores):
    print("F1 Score for label {}: {:.2f}".format(label, f1_score))

In [ ]:
# support 0.40 deny 0.55 comment 0.90 query 0.22 0.385
# support 0.02 commnet 0.34 deny 0.91 query 0.48
# 'support': 0, 'comment': 1, 'deny': 2, 'query': 3}

In [ ]:
modified_model.save("lata", save_format="tf")


In [ ]:
# from tensorflow.keras.models import load_model



# # Load the model
# loaded_model = load_model("lata")

# # Now you can use the loaded_model for inference without retraining


In [ ]:
predictions=modified_model.predict(X_test)

In [ ]:
predictions

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Assuming y_true and y_pred are NumPy arrays
# If they are not NumPy arrays, you can convert them using np.array(your_list)

# Example true labels (ground truth) for a multiclass problem
y_true = np.array(label_test)

# Example predicted probabilities from your model
# This assumes you have a model predicting probabilities for each class
y_probs = np.array(predictions)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_probs, axis=-1)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}')

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Assuming y_true and y_pred are NumPy arrays
# If they are not NumPy arrays, you can convert them using np.array(your_list)

# Example true labels (ground truth) for a multiclass problem
y_true = np.array(label_test)

# Example predicted probabilities from your model
# This assumes you have a model predicting probabilities for each class
y_probs = np.array(predictions)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_probs, axis=-1)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}')

# Calculate precision, recall, and F1-score (weighted)
precision_weighted = precision_score(y_true, y_pred, average='weighted')
recall_weighted = recall_score(y_true, y_pred, average='weighted')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print(f'Precision (Weighted): {precision_weighted}')
print(f'Recall (Weighted): {recall_weighted}')
print(f'F1-Score (Weighted): {f1_weighted}')

# Calculate precision, recall, and F1-score (macro)
precision_macro = precision_score(y_true, y_pred, average='macro')
recall_macro = recall_score(y_true, y_pred, average='macro')
f1_macro = f1_score(y_true, y_pred, average='macro')

print(f'Precision (Macro): {precision_macro}')
print(f'Recall (Macro): {recall_macro}')
print(f'F1-Score (Macro): {f1_macro}')


In [ ]:
with open("metrics.txt", "w") as file:
    file.write(f'Accuracy: {accuracy}\n')
    file.write(f'Precision: {precision}\n')
    file.write(f'Recall: {recall}\n')
    file.write(f'F1-Score: {f1}\n')

print("Metrics saved to metrics.txt")